In [3]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import date
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from lxml import etree
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, median_absolute_error, explained_variance_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from matplotlib.dates import AutoDateLocator, AutoDateFormatter
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import layers, models, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
import graphviz
from IPython.display import Image
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.ensemble import IsolationForest

In [ ]:
houses = {}
house_names = []
for i in range(1, 22):
    # number 14 is missing from the dataset
    if i == 14:
        continue
    file_name = f'CLEAN_House{i}.csv'
    try:
        # path_to_Houses.csv
        df = pd.read_csv(file_name, parse_dates=['Time'])
        # Add the dataframe to the list
        houses[file_name] = df
        #houses.append(df)
        house_names.append(file_name)
        print(file_name)
    except FileNotFoundError:
        print(f'File not found: {file_name}')
        continue

In [ ]:
import pandas as pd

# Load weather data
weather_df = pd.read_csv('weather.csv', parse_dates=['date'])

# Convert weather data timestamps to match house timestamps format (if necessary)
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%d/%m/%Y %H:%M')  # Assuming weather data format is 'dd/mm/yyyy hh:mm'

# Dictionary to hold the houses dataframes
houses2 = {}
house_names2 = []

# Loop through house files, skipping number 14
for i in range(1, 21):
        continue
    file_name = f'H{i}_W.csv'
    
    try:
        # Load house data
        house_df2 = pd.read_csv(f'{file_name}', parse_dates=['date'])
        
        # Merge the house data with weather data on the 'date' column (inner join to keep only matching timestamps)
        merged_df = pd.merge(house_df, weather_df, on='date', how='inner')
        
        # Add the merged dataframe to the dictionary
        houses2[file_name] = merged_df
        house_names2.append(file_name)
        
        # Display the first few rows of the merged dataframe to verify
        print(f"Merged data for {file_name}:")
        print(houses2[file_name].head())
        
    except FileNotFoundError:
        print(f'File not found: {file_name}')
        continue


In [9]:
def detect_anomalies_zscore(data, window_size=30, z_threshold=3):
    # Compute rolling mean and standard deviation
    rolling_mean = data.rolling(window=window_size).mean()
    rolling_std = data.rolling(window=window_size).std()
    
    # Calculate Z-Score
    z_scores = (data - rolling_mean) / rolling_std
    anomalies = np.abs(z_scores) > z_threshold
    
    # Plot the data with anomalies
    plt.figure(figsize=(20, 6))
    plt.plot(data.index, data, label='Original Data', alpha=0.6)
    plt.scatter(data.index[anomalies], data[anomalies], color='red', label='Anomalies', marker='o')
    plt.title('Anomaly Detection using Z-Score')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_ZScore.jpeg', format='jpeg', dpi=300)
    plt.show()
    return anomalies

def detect_anomalies_moving_average(data, window_size=30, threshold_factor=3):
    rolling_mean = data.rolling(window=window_size).mean()
    rolling_std = data.rolling(window=window_size).std()
    
    # Set upper and lower bounds
    upper_bound = rolling_mean + threshold_factor * rolling_std
    lower_bound = rolling_mean - threshold_factor * rolling_std
    anomalies = (data > upper_bound) | (data < lower_bound)
    
    plt.figure(figsize=(20, 6))
    plt.plot(data.index, data, label='Original Data', alpha=0.6)
    plt.plot(data.index, upper_bound, color='green', label='Upper Bound')
    plt.plot(data.index, lower_bound, color='green', label='Lower Bound')
    plt.scatter(data.index[anomalies], data[anomalies], color='red', label='Anomalies', marker='o')
    plt.title('Anomaly Detection using Moving Average')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Moving_Average.jpeg', format='jpeg', dpi=300)
    plt.show()

    return anomalies


def detect_anomalies_isolation_forest(data, contamination=0.01):
    # Fit the Isolation Forest
    iso_forest = IsolationForest(contamination=contamination)
    data = data.dropna().to_frame()
    data['Anomaly_Score'] = iso_forest.fit_predict(data)
    anomalies = data['Anomaly_Score'] == -1
    plt.figure(figsize=(20, 6))
    plt.plot(data.index, data.iloc[:, 0], label='Original Data', alpha=0.6)
    plt.scatter(data.index[anomalies], data.iloc[:, 0][anomalies], color='red', label='Anomalies', marker='o')
    plt.title('Anomaly Detection using Isolation Forest')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Isolation_Forest.jpeg', format='jpeg', dpi=300)
    plt.show()
    return anomalies



In [ ]:
for name, df in houses.items():
    print(f'Processing {name}...')
    
    df['Aggregate'] = df['Aggregate'].replace(0, 1e-6)
    zero_values_after_replacement = (df['Aggregate'] == 0).sum()
    print(zero_values_after_replacement)
    df['Aggregate'] = df['Aggregate'].fillna((df['Aggregate'].ffill() + df['Aggregate'].bfill()) / 2)
    df['Aggregate'] = df['Aggregate'].bfill()
    df['Aggregate'] = df['Aggregate'].ffill()
    df['Time'] = pd.to_datetime(df['Time'])
    df.set_index('Time', inplace=True)
    
    plt.figure(figsize=(15, 7))
    plt.plot(df.index, df['Aggregate'], label='Momentary')
    plt.title('Momentary Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.show()

    hourly_data = df['Aggregate'].resample('H').mean()
    hourly_data = hourly_data.bfill()
    hourly_data = hourly_data.ffill()
    plt.figure(figsize=(15, 7))
    plt.plot(hourly_data.index, hourly_data, label='Hourly')
    plt.title('Hourly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Hourly.jpeg', format='jpeg', dpi=300)
    plt.show()

    daily_data = df['Aggregate'].resample('D').mean()
    daily_data = daily_data.bfill()
    daily_data = daily_data.ffill()
    
    plt.figure(figsize=(15, 7))
    plt.plot(daily_data.index, daily_data, label='Daily')
    plt.title('Daily Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Daily.jpeg', format='jpeg', dpi=300)
    plt.show()

    weekly_data = df['Aggregate'].resample('W').mean()
    weekly_data = weekly_data.bfill()
    weekly_data = weekly_data.ffill()

    plt.figure(figsize=(15, 7))
    plt.plot(weekly_data.index, weekly_data, label='Weekly')
    plt.title('Weekly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Weekly.jpeg', format='jpeg', dpi=300)
    plt.show()

    monthly_data = df['Aggregate'].resample('M').mean()
    monthly_data = monthly_data.bfill()
    monthly_data = monthly_data.ffill()
    plt.figure(figsize=(15, 7))
    plt.plot(monthly_data.index, monthly_data, label='Monthly')
    plt.title('Monthly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Monthly.jpeg', format='jpeg', dpi=300)
    plt.show()
    
    decomposition = seasonal_decompose(df['Aggregate'], model='additive', period=1)
    plt.figure(figsize=(14, 8))
    decomposition.plot()
    plt.suptitle(f'Daily Time Series Decomposition for {name}', y=1.02)
    plt.tight_layout()
    plt.show()
    decomposition = seasonal_decompose(daily_data, model='additive', period=7)
    plt.figure(figsize=(14, 8))
    decomposition.plot()
    plt.suptitle(f'Daily Time Series Decomposition for {name}', y=1.02)
    plt.tight_layout()
    plt.savefig(f'{name}_Decomposition.jpeg', format='jpeg', dpi=300)
    plt.show()

    detect_anomalies_zscore(df['Aggregate'], window_size=30, z_threshold=3)  
    detect_anomalies_zscore(hourly_data, window_size=24, z_threshold=3)  
    detect_anomalies_zscore(daily_data, window_size=7, z_threshold=3) 
    detect_anomalies_zscore(weekly_data, window_size=4, z_threshold=3)

    detect_anomalies_moving_average(df['Aggregate'], window_size=30, threshold_factor=3)
    detect_anomalies_moving_average(hourly_data, window_size=24, threshold_factor=3) 
    detect_anomalies_moving_average(daily_data, window_size=7, threshold_factor=3) 
    detect_anomalies_moving_average(weekly_data, window_size=4, threshold_factor=3) 

    detect_anomalies_isolation_forest(df['Aggregate'], contamination=0.01) 
    detect_anomalies_isolation_forest(hourly_data, contamination=0.01)
    detect_anomalies_isolation_forest(daily_data, contamination=0.01) 
    detect_anomalies_isolation_forest(weekly_data, contamination=0.01)
        
    print(f"Creating features with interpolation for {name}...")
    df.interpolate(method='linear', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df = pd.DataFrame(data)
    file_path = f'{name}.csv'
    df.to_csv(file_path, index=False)


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

for name, df in houses2.items():
    print(f'Processing {name}...')
    
    # Replace 0 values in the 'Aggregate' column with a small value (1e-6)
    df['Aggregate'] = df['Aggregate'].replace(0, 1e-6)
    zero_values_after_replacement = (df['Aggregate'] == 0).sum()
    print(zero_values_after_replacement)
    df['Aggregate'] = df['Aggregate'].fillna((df['Aggregate'].ffill() + df['Aggregate'].bfill()) / 2)
    df['Aggregate'] = df['Aggregate'].bfill()
    df['Aggregate'] = df['Aggregate'].ffill()
    df['Time'] = pd.to_datetime(df['Time'])
    df.set_index('Time', inplace=True)
    
    plt.figure(figsize=(15, 7))
    plt.plot(df.index, df['Aggregate'], label='Momentary')
    plt.title('Momentary Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.show()

    hourly_data= df['Aggregate'].resample('H').mean()
    hourly_data = hourly_data.bfill()
    hourly_data = hourly_data.ffill()
    plt.figure(figsize=(15, 7))
    plt.plot(hourly_data.index, hourly_data, label='Hourly')
    plt.title('Hourly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Hourly.jpeg', format='jpeg', dpi=300)
    plt.show()

    daily_data = df['Aggregate'].resample('D').mean()
    daily_data = daily_data.bfill()
    daily_data = daily_data.ffill()
    
    plt.figure(figsize=(15, 7))
    plt.plot(daily_data.index, daily_data, label='Daily')
    plt.title('Daily Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Daily.jpeg', format='jpeg', dpi=300)
    plt.show()

    weekly_data = df['Aggregate'].resample('W').mean()
    weekly_data = weekly_data.bfill()
    weekly_data = weekly_data.ffill()

    plt.figure(figsize=(15, 7))
    plt.plot(weekly_data.index, weekly_data, label='Weekly')
    plt.title('Weekly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Weekly.jpeg', format='jpeg', dpi=300)
    plt.show()

    monthly_data = df['Aggregate'].resample('M').mean()
    monthly_data = monthly_data.bfill()
    monthly_data = monthly_data.ffill()
    plt.figure(figsize=(15, 7))
    plt.plot(monthly_data.index, monthly_data, label='Monthly')
    plt.title('Monthly Energy Consumption')
    plt.xlabel('Time')
    plt.ylabel('Energy (W)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{name}_Monthly.jpeg', format='jpeg', dpi=300)
    plt.show()
    
    decomposition = seasonal_decompose(df['Aggregate'], model ='additive', period = 1)
    plt.figure(figsize=(14, 8))
    decomposition.plot()
    plt.suptitle(f'Daily Time Series Decomposition for {name}', y=1.02)
    plt.tight_layout()
    plt.show()
    decomposition = seasonal_decompose(daily_data, model='additive', period=7)  #
    plt.figure(figsize=(14, 8))
    decomposition.plot()
    plt.suptitle(f'Daily Time Series Decomposition for {name}', y=1.02)
    plt.tight_layout()
    plt.savefig(f'{name}_Decomposition.jpeg', format='jpeg', dpi=300)
    plt.show()

    detect_anomalies_zscore(df['Aggregate'], window_size=30, z_threshold=3)  
    detect_anomalies_zscore(hourly_data, window_size=24, z_threshold=3)  
    detect_anomalies_zscore(daily_data, window_size=7, z_threshold=3) 
    detect_anomalies_zscore(weekly_data, window_size=4, z_threshold=3)

    detect_anomalies_moving_average(df['Aggregate'], window_size=30, threshold_factor=3)
    detect_anomalies_moving_average(hourly_data, window_size=24, threshold_factor=3) 
    detect_anomalies_moving_average(daily_data, window_size=7, threshold_factor=3) 
    detect_anomalies_moving_average(weekly_data, window_size=4, threshold_factor=3) 

    detect_anomalies_isolation_forest(df['Aggregate'], contamination=0.01) 
    detect_anomalies_isolation_forest(hourly_data, contamination=0.01)
    detect_anomalies_isolation_forest(daily_data, contamination=0.01) 
    detect_anomalies_isolation_forest(weekly_data, contamination=0.01)
        
    
    print(f"Creating features with interpolation for {name}...")
    df.interpolate(method='linear', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df = pd.DataFrame(data)
    file_path = f'{name}.csv'
    df.to_csv(file_path, index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Processing each house dataset and performing EDA on weather features
for name, df in houses.items():
    print(f'Processing {name}...')
    
    # Strip leading/trailing spaces from column names
    df.columns = df.columns.str.strip()

    # Replace 0 values in 'Consumption(W)' and 'Production(W)' with a small value (1e-6)
    df['Consumption(W)'] = df['Consumption(W)'].replace(0, 1e-6)
    df['Production(W)'] = df['Production(W)'].replace(0, 1e-6)
    
    # Fill missing values for energy columns
    df['Consumption(W)'] = df['Consumption(W)'].fillna((df['Consumption(W)'].ffill() + df['Consumption(W)'].bfill()) / 2)
    df['Production(W)'] = df['Production(W)'].fillna((df['Production(W)'].ffill() + df['Production(W)'].bfill()) / 2)

    # Fill missing weather data and ensure numeric values
    weather_columns = ['speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']
    for column in weather_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')  # Convert to numeric, replacing non-numeric with NaN
        df[column].fillna(df[column].mean(), inplace=True)  # Fill missing values with column mean

    # Perform correlation analysis on cleaned data
    numeric_columns = ['Consumption(W)', 'Production(W)', 'speed', 'drybulb', 'soltot', 'rain']
    correlation_matrix = df[numeric_columns].corr()

    # Plot a heatmap of the correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title(f'Correlation Matrix for {name} (Energy and Weather Features)')
    plt.tight_layout()
    plt.savefig(f'{name}_Correlation_Matrix.jpeg', format='jpeg', dpi=300)
    plt.show()

    # SCATTER PLOTS (Consumption and Production vs Weather Features)
    
    # Scatter plot: Energy Consumption vs Temperature (drybulb)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['drybulb'], df['Consumption(W)'], alpha=0.5)
    plt.title(f'Energy Consumption vs Temperature (Dry Bulb) for {name}')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Consumption (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Consumption_vs_Temperature.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Production vs Temperature (drybulb)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['drybulb'], df['Production(W)'], alpha=0.5, color='orange')
    plt.title(f'Energy Production vs Temperature (Dry Bulb) for {name}')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Production (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Production_vs_Temperature.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Consumption vs Wind Speed (speed)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['speed'], df['Consumption(W)'], alpha=0.5, color='purple')
    plt.title(f'Energy Consumption vs Wind Speed for {name}')
    plt.xlabel('Wind Speed (m/s)')
    plt.ylabel('Consumption (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Consumption_vs_Wind_Speed.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Production vs Wind Speed (speed)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['speed'], df['Production(W)'], alpha=0.5, color='green')
    plt.title(f'Energy Production vs Wind Speed for {name}')
    plt.xlabel('Wind Speed (m/s)')
    plt.ylabel('Production (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Production_vs_Wind_Speed.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Consumption vs Solar Radiation (soltot)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['soltot'], df['Consumption(W)'], alpha=0.5, color='blue')
    plt.title(f'Energy Consumption vs Solar Radiation for {name}')
    plt.xlabel('Solar Radiation (W/m²)')
    plt.ylabel('Consumption (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Consumption_vs_Solar_Radiation.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Production vs Solar Radiation (soltot)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['soltot'], df['Production(W)'], alpha=0.5, color='orange')
    plt.title(f'Energy Production vs Solar Radiation for {name}')
    plt.xlabel('Solar Radiation (W/m²)')
    plt.ylabel('Production (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Production_vs_Solar_Radiation.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Consumption vs Rain (rain)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['rain'], df['Consumption(W)'], alpha=0.5, color='yellow')
    plt.title(f'Energy Consumption vs Rain for {name}')
    plt.xlabel('Rainfall (mm)')
    plt.ylabel('Consumption (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Consumption_vs_Rain.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Scatter plot: Energy Production vs Rain (rain)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['rain'], df['Production(W)'], alpha=0.5, color='red')
    plt.title(f'Energy Production vs Rain for {name}')
    plt.xlabel('Rainfall (mm)')
    plt.ylabel('Production (W)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}_Production_vs_Rain.jpeg', format='jpeg', dpi=300)
    plt.show()
    
    # BINNED PLOTS (Boxplots)
    
  # Binned plot: Temperature vs Consumption
    df['temp_bins'] = pd.cut(df['drybulb'], bins=10)
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['temp_bins'], y=df['Consumption(W)'], palette='Blues')
    plt.title(f'Binned Temperature vs Energy Consumption for {name}')
    plt.xlabel('Temperature Bins')
    plt.ylabel('Consumption (W)')
    plt.grid(True, color='blue', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Temperature_vs_Consumption.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Binned plot: Temperature vs Production
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['temp_bins'], y=df['Production(W)'], palette='Oranges')
    plt.title(f'Binned Temperature vs Energy Production for {name}')
    plt.xlabel('Temperature Bins')
    plt.ylabel('Production (W)')
    plt.grid(True, color='blue', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Temperature_vs_Production.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Binned plot: Wind Speed vs Consumption
    df['wind_bins'] = pd.cut(df['speed'], bins=10)
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['wind_bins'], y=df['Consumption(W)'], palette='Purples')
    plt.title(f'Binned Wind Speed vs Energy Consumption for {name}')
    plt.xlabel('Wind Speed Bins (m/s)')
    plt.ylabel('Consumption (W)')
    plt.grid(True, color='green', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Wind_Speed_vs_Consumption.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Binned plot: Wind Speed vs Production
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['wind_bins'], y=df['Production(W)'], palette='Greens')
    plt.title(f'Binned Wind Speed vs Energy Production for {name}')
    plt.xlabel('Wind Speed Bins (m/s)')
    plt.ylabel('Production (W)')
    plt.grid(True, color='green', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Wind_Speed_vs_Production.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Binned plot: Solar Radiation vs Consumption
    df['solar_bins'] = pd.cut(df['soltot'], bins=10)
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['solar_bins'], y=df['Consumption(W)'], palette='Blues')
    plt.title(f'Binned Solar Radiation vs Energy Consumption for {name}')
    plt.xlabel('Solar Radiation Bins (W/m²)')
    plt.ylabel('Consumption (W)')
    plt.grid(True, color='orange', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Solar_Radiation_vs_Consumption.jpeg', format='jpeg', dpi=300)
    plt.show()

    # Binned plot: Solar Radiation vs Production
    plt.figure(figsize=(20, 10))
    sns.boxplot(x=df['solar_bins'], y=df['Production(W)'], palette='Oranges')
    plt.title(f'Binned Solar Radiation vs Energy Production for {name}')
    plt.xlabel('Solar Radiation Bins (W/m²)')
    plt.ylabel('Production (W)')
    plt.grid(True, color='orange', linestyle='--', linewidth=1)  # Change the grid line color and style
    plt.tight_layout()
    plt.savefig(f'{name}_Binned_Solar_Radiation_vs_Production.jpeg', format='jpeg', dpi=300)
    plt.show()
